<a href="https://colab.research.google.com/github/rahulbhethanabotla/CS8803HMLFinalProject/blob/main/Amit_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

Put imports for all frameworks, libraries, and pip installs here and run this before any further sections.

In [1]:
!pip install vaderSentiment
!pip install transformers[torch]
!pip install gensim
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.5 MB/s 
     |████████████████████████████████| 7.6 MB 8.5 MB/s 
     |████████████████████████████████| 182 kB 27.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (943/943), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 3930 (delta 854), reused 806 (delta 806), pack-reused 2987
Receiving objects: 100% (3930/3930), 8.24 MiB | 25.12 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
make: *** No targets specified and no makefile found.  Stop.


In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from scipy import spatial
import numpy as np
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, BertTokenizer, BertModel
import torch
import time
import csv
from tqdm import tqdm_notebook
import io
from gensim.models import FastText as ft
import fastText

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Embedding Models (BERT, GPT2, FastText)

## GPT2

In [4]:
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')  # or any other checkpoint
word_embeddings = gpt2_model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = gpt2_model.transformer.wpe.weight  # Word Position Embeddings 
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [5]:
def get_GPT2_word_embedding(word):
  text_index = gpt2_tokenizer.encode(word, add_prefix_space=True)
  vector = gpt2_model.transformer.wte.weight[text_index,:]
  return vector.detach().numpy()

In [6]:
type(get_GPT2_word_embedding("dog"))

numpy.ndarray

## FastText

In [7]:
ft_model=ft.load_fasttext_format("/content/drive/MyDrive/CS8803HMLFinalProject/pretrained_embeddings/cc.en.300.bin")

In [8]:
def get_FT_word_embedding(word):
  return ft_model.wv[word]

In [9]:
type(get_FT_word_embedding("dog"))

numpy.ndarray

## BERT Word Embeddings

In [10]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def get_BERT_word_embedding(text):
  marked_text = "[CLS] " + text + " [SEP]"

  # Tokenize our sentence with the BERT tokenizer.
  tokenized_text = bert_tokenizer.tokenize(marked_text)
  indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  bert_model.eval()
  with torch.no_grad():

      outputs = bert_model(tokens_tensor, segments_tensors)

      # Evaluating the model will return a different number of objects based on 
      # how it's  configured in the `from_pretrained` call earlier. In this case, 
      # becase we set `output_hidden_states = True`, the third item will be the 
      # hidden states from all layers. See the documentation for more details:
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2]
  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)
  token_vecs_cat = []
  for token in token_embeddings:
      
      # `token` is a [12 x 768] tensor

      # Concatenate the vectors (that is, append them together) from the last 
      # four layers.
      # Each layer vector is 768 values, so `cat_vec` is length 3,072.
      cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
      
      # Use `cat_vec` to represent `token`.
      token_vecs_cat.append(cat_vec)

  return token_vecs_cat[1].detach().numpy()

In [12]:
get_BERT_word_embedding("dog")

array([-0.09447087,  0.20714663, -0.3122165 , ..., -0.64669365,
        0.0215818 , -0.55317575], dtype=float32)

# VADER Sentiment Analysis

In [13]:
# Start VADER

analyzer = SentimentIntensityAnalyzer()
word = 'search find'
vs = analyzer.polarity_scores(word)
print("{:-<65} {}".format(word, str(vs)))

search find------------------------------------------------------ {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [14]:
# Converts text to a SA vector through VADER

def text_2_sentiment_vec(text):
  vs = analyzer.polarity_scores(text)
  sent_vec = np.zeros((len(vs)))
  for idx, key in enumerate(vs):
    sent_vec[idx] = vs[key]

  return sent_vec

In [15]:
text_2_sentiment_vec("funeral somber")

array([ 1.    ,  0.    ,  0.    , -0.6486])

In [16]:
# Testing analogy pairs with VADER

analyzer = SentimentIntensityAnalyzer()

questions = [["diamond:baseball", "court:poker", "court:jury", "court:grass", "court:squash", 4],
             ["bench:judge", "throne:king", "queen:king", "court:king", "knight:king", 3],
             ["funeral:somber", "tension:festive", "soiree:festive", "eulogy:festive", "sari:festive", 2],
             ["defeat:vanquish", "search:peer", "search:ransack", "search:destroy", "search:find", 4],
             ["slug:land", "shark:seaweed", "shark:ocean", "shark:sky", "shark:slide", 2]]

analogy_predictions = []

for p in range(0, len(questions)):
  print("Q"+ str(p+1) +":")
  q = questions[p]
  
  a = q[0].split(':')
  a_combined = a[0] + ' ' + a[1]
  a_vec = text_2_sentiment_vec(a_combined)
  print(a_vec)

  sim_scores = []
  for i in range(1,5):
    b = q[i].split(':')
    b_combined = b[0] + ' ' + b[1]
    b_vec = text_2_sentiment_vec(b_combined)
    print(b_vec)

    sim = 1 - spatial.distance.cosine(a_vec, b_vec)
    sim_scores.append(sim)

  print("Similarities:")
  print(sim_scores)
  analogy_predictions.append(np.argmax(sim_scores)+1)
  print()
  



print("-----")

correct_anal = 0
for i in range (0,len(questions)):
  if questions[i][5] == analogy_predictions[i]:
    print("Correct answer for Question #" + str(i+1))
  else:
    print("Incorrect answer for Question #" + str(i+1))

Q1:
[0.    0.294 0.706 0.34 ]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
Similarities:
[0.3512787999634166, 0.3512787999634166, 0.3512787999634166, 0.3512787999634166]

Q2:
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
Similarities:
[1, 1, 1, 1]

Q3:
[ 1.      0.      0.     -0.6486]
[0.434  0.     0.566  0.1779]
[0.     0.25   0.75   0.4588]
[0.     0.25   0.75   0.4588]
[0.     0.25   0.75   0.4588]
Similarities:
[0.3636425902019753, -0.27313626031938254, -0.27313626031938254, -0.27313626031938254]

Q4:
[ 0.75    0.25    0.     -0.4588]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[ 0.778   0.222   0.     -0.5423]
[0. 1. 0. 0.]
Similarities:
[0.2735063708444414, 0.2735063708444414, 0.997221070807447, 0.2735063708444414]

Q5:
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
Similarities:
[1, 1, 1, 1]

-----
Incorrect answer for Question #1
Incorrect answer for Question #2
Incorrect answer for Question #3
Incorrect answer for Question #4
Incorre

In [ ]:
# Testing with synonyms/antonyms

# Need CSV from Project 3
syntest = pd.read_csv("/content/drive/MyDrive/CS8803HMLFinalProject/pretrained_embeddings/syntest.csv")
analyzer = SentimentIntensityAnalyzer()

prediction_list = []

for k in range(0,len(syntest)):
  t = syntest.loc[k]
  q = text_2_sentiment_vec(t.loc["Question"])
  sim_scores = []
  for i in range(1,5):
    sim = 1 - spatial.distance.cosine(q, text_2_sentiment_vec(t.loc["Answer"+str(i)]))
    sim_scores.append(sim)

  # Use print to see all computed similarity scores
  #print(k+1, sim_scores)
  if t.loc["Type"] == 'synonym':
    prediction = t.loc["Answer"+str(np.argmax(sim_scores) + 1)]
  else:
    prediction = t.loc["Answer"+str(np.argmin(sim_scores) + 1)]

  if prediction == t.loc["Correct"]:
    prediction_list.append(1)
  else:
    prediction_list.append(0)

print("Correct answers: " + str(np.count_nonzero(prediction_list)) +"/"+ str(len(prediction_list)) )

# Parallelogram model

In [20]:
def get_embedding(word, embed_type):
  word_embed = ""
  if embed_type == "BERT":
    word_embed = get_BERT_word_embedding(word)
  elif embed_type == "GPT2":
    word_embed = get_GPT2_word_embedding(word)
  elif embed_type == "FT":
    word_embed = get_FT_word_embedding(word)

  return word_embed

In [41]:
def evaluate(embed_type):
  # time.asctime( time.localtime(time.time()) )

  curr_time = time.time
  data = pd.read_csv("/content/drive/MyDrive/CS8803HMLFinalProject/pretrained_embeddings/Turney_data.csv")

  rows_sim = []

  opt_list = ["a", "b", "c", "d", "e"]

  num_corr = 0

  for index, row in data.iterrows():
    rows_ = [str(index)]
    qs1, qs2 = str(row["Qs"]).split(":")
    # print(qs1)
    # print(get_embedding(qs1, embed_type).shape)
    
    qs_diff = get_embedding(qs1, embed_type) - get_embedding(qs2, embed_type)
    # print(qs_diff.shape)

    ans_scores = []
    for opt in opt_list:
      ans1, ans2 = str(row[opt]).split(":")
      # print(ans1)
      # print(get_embedding(ans1, embed_type).shape)
      # print(ans2)
      # print(get_embedding(ans2, embed_type).shape)
      ans_diff = get_embedding(ans1, embed_type) - get_embedding(ans2, embed_type)
      # print(ans_diff.shape)
      sim = 1 - spatial.distance.cosine(qs_diff, ans_diff)
      rows_.append(sim)
      ans_scores.append(sim)
    
    pred_ans = opt_list[ np.argmax(ans_scores) ]
    rows_.append(str(row["Ans"]))
    rows_.append(str(pred_ans))

    if str(row["Ans"]) == pred_ans:
      num_corr += 1

    rows_sim.append(rows_)
  
  header = ["Qs", "a", "b", "c", "d", "e", "Corr_Ans", "Pred_Ans"]
  with open('/content/drive/MyDrive/CS8803HMLFinalProject/results/results_' + str(time.time()) + '.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(rows_sim)

  print("Accuracy: " + str(num_corr) + "/" + str(374))

In [43]:
evaluate("FT")

Accuracy: 169/374
